In [ ]:
import numpy as np


def create_training_data():
    data = [
        ["Sunny", "Hot", "High", "Weak", "no"],
        ["Sunny", "Hot", "High", "Strong", "no"],
        ["Overcast", "Hot", "High", "Weak", "yes"],
        ["Rain", "Mild", "High", "Weak", "yes"],
        ["Rain", "Cool", "Normal", "Weak", "yes"],
        ["Rain", "Cool", "Normal", "Strong", "no"],
        ["Overcast", "Cool", "Normal", "Strong", "yes"],
        ["Overcast", "Mild", "High", "Weak", "no"],
        ["Sunny", "Cool", "Normal", "Weak", "yes"],
        ["Rain", "Mild", "Normal", "Weak", "yes"],
    ]
    return np.array(data)


train_data = create_training_data()
print(train_data)
print(train_data.shape)

[['Sunny' 'Hot' 'High' 'Weak' 'no']
 ['Sunny' 'Hot' 'High' 'Strong' 'no']
 ['Overcast' 'Hot' 'High' 'Weak' 'yes']
 ['Rain' 'Mild' 'High' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'no']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'yes']
 ['Overcast' 'Mild' 'High' 'Weak' 'no']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'yes']]
(10, 5)
['Rain' 'Cool' 'Normal' 'Weak' 'yes']


In [ ]:
def compute_prior_probabilities(train_data):
    class_names = ["no", "yes"]
    total_samples = len(train_data)
    prior_probs = np.zeros(len(class_names))
    # your code here
    labels = train_data[:, -1]
    for i, class_name in enumerate(class_names):
        count = np.sum(labels == class_name)
        prior_probs[i] = count / total_samples

    return prior_probs


prior_probability = compute_prior_probabilities(train_data)
print("P('Play Tennis' = No)", prior_probability[0])
print("P('Play Tennis' = Yes)", prior_probability[1])

P('Play Tennis' = No) 0.4
P('Play Tennis' = Yes) 0.6


In [ ]:
def compute_conditional_probabilities(train_data):
    class_names = ["no", "yes"]
    n_features = train_data.shape[1] - 1  # Exclude target column
    conditional_probs = []
    feature_values = []
    labels = train_data[:, -1]

    for feature_idx in range(n_features):
        unique_values = np.unique(train_data[:, feature_idx])
        feature_values.append(unique_values)

        feature_cond_probs = np.zeros((len(class_names), len(unique_values)))

        for class_idx, class_name in enumerate(class_names):
            # get samples for this class
            samples_in_class = train_data[labels == class_name]
            feature_col_in_class = samples_in_class[:, feature_idx]
            count_class = np.sum(labels == class_name)

            for value_idx, value in enumerate(unique_values):
                # count occurrences of this feature value in this class
                count_val_in_class = np.sum(feature_col_in_class == value)
                # calculate conditional probability
                conditinal_probability = count_val_in_class / count_class

                feature_cond_probs[class_idx, value_idx] = conditinal_probability
        conditional_probs.append(feature_cond_probs)

    return conditional_probs, feature_values


# Test
_, feature_values = compute_conditional_probabilities(train_data)
print(_)
print("x1 = ", feature_values[0])
print("x2 = ", feature_values[1])
print("x3 = ", feature_values[2])
print("x4 = ", feature_values[3])

[array([[0.25      , 0.25      , 0.5       ],
       [0.33333333, 0.5       , 0.16666667]]), array([[0.25      , 0.5       , 0.25      ],
       [0.5       , 0.16666667, 0.33333333]]), array([[0.75      , 0.25      ],
       [0.33333333, 0.66666667]]), array([[0.5       , 0.5       ],
       [0.16666667, 0.83333333]])]
x1 =  ['Overcast' 'Rain' 'Sunny']
x2 =  ['Cool' 'Hot' 'Mild']
x3 =  ['High' 'Normal']
x4 =  ['Strong' 'Weak']


In [ ]:
def get_feature_index(feature_value, feature_values):
    index_array = np.where(feature_values == feature_value)
    if len(index_array[0] > 0):
        return index_array[0][0]
    else:
        return -1


train_data = create_training_data()
_, feature_values = compute_conditional_probabilities(train_data)

outlook = feature_values[0]

i1 = get_feature_index("Overcast", outlook)
i2 = get_feature_index("Rain", outlook)
i3 = get_feature_index("Sunny", outlook)

print(f"Index of 'Overcast': {i1}")
print(f"Index of 'Rain': {i2}")
print(f"Index of 'Sunny': {i3}")
print(f"Combined print: {i1}, {i2}, {i3}")

Index of 'Overcast': 0
Index of 'Rain': 1
Index of 'Sunny': 2
Combined print: 0, 1, 2


In [ ]:
def train_naive_bayes(train_data):
    prior_probabilities = compute_prior_probabilities(train_data)

    conditional_probabilites, feature_names = compute_conditional_probabilities(
        train_data
    )

    return prior_probabilities, conditional_probabilites, feature_names


prior_probs, conditional_probs, feature_names = train_naive_bayes(train_data)

print(prior_probs, conditional_probs, feature_names)

[0.4 0.6] [array([[0.25      , 0.25      , 0.5       ],
       [0.33333333, 0.5       , 0.16666667]]), array([[0.25      , 0.5       , 0.25      ],
       [0.5       , 0.16666667, 0.33333333]]), array([[0.75      , 0.25      ],
       [0.33333333, 0.66666667]]), array([[0.5       , 0.5       ],
       [0.16666667, 0.83333333]])] [array(['Overcast', 'Rain', 'Sunny'], dtype='<U8'), array(['Cool', 'Hot', 'Mild'], dtype='<U8'), array(['High', 'Normal'], dtype='<U8'), array(['Strong', 'Weak'], dtype='<U8')]


In [ ]:
def predict_tennis(X, prior_probabilities, conditional_probabilites, feature_names):
    class_names = ["no", "yes"]
    feature_indices = []
    for i, feature_value in enumerate(X):
        feature_indices.append(get_feature_index(feature_value, feature_names[i]))

    class_probabilities = []

    for class_idx in range(len(class_names)):
        # start with prior probability
        current_class_prob = prior_probabilities[class_idx]
        # Multiply by conditional probabilities
        for feature_idx_in_X, val_index in enumerate(feature_indices):
            cond_prob = conditional_probabilites[feature_idx_in_X][class_idx, val_index]
            current_class_prob *= cond_prob
        class_probabilities.append(current_class_prob)

    total_prob = sum(class_probabilities)
    if total_prob > 0:
        normalized_probs = [p / total_prob for p in class_probabilities]
    else:
        normalized_probs = [0.5, 0.5]

    predicted_class_idx = np.argmax(class_probabilities)
    prediction = class_names[predicted_class_idx]

    prob_dict = {
        "no": round(normalized_probs[0].item(), 2),
        "yes": round(normalized_probs[1].item(), 2),
    }

    return prediction, prob_dict


X = ["Sunny", "Cool", "High", "Strong"]
prior_probs, conditional_probs, feature_names = train_naive_bayes(train_data)
prediction, prob_dict = predict_tennis(X, prior_probs, conditional_probs, feature_names)
print(prediction)
print(prob_dict)

no
{'no': 0.87, 'yes': 0.13}
